In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error


# S1: Extract the required attribute columns and save them as "total_inference_X"

total_inference_df = pd.read_csv("./inputs/Grid feature statistics.csv")
# region partition
# dummy_df = pd.get_dummies(total_inference_df["Partition"])
# partition_df = pd.get_dummies(total_inference_df['Partition'], prefix='Partition')  
# total_inference_df = pd.concat([total_inference_df, partition_df], axis=1)

total_inference_X = total_inference_df.drop(columns=['code', 'flag', 'Lon', 'Lat', 'Rooftop_area',
                                                     'City', 'Partition', 'Level', 'Bare_ratio',
                                                     'Dem_average',
                                                     'Dem_difference',
                                                     'Slope_average',
                                                     'Snow_ratio', 'Trees_ratio', 'Water_ratio'])

# print(total_inference_X)
total_inference_X.to_csv(r'./reoutputs/s1_re_total_inference_X.csv',index=None)

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

#S2:根据Grid feature statistics.csv中 flag=1的city（86个真实值）提取出来，并将其值赋予total_inference_X中flag=1的行中；
#同时，根据86个城市的建筑物屋顶数据集训练，然后训练的随机森林外推到其他354个城市中

def max_min_scale(X, mins=None, maxs=None):
    return (X-mins) / (maxs-mins)

total_inference_df = pd.read_csv("./inputs/Grid feature statistics.csv")
total_inference_X = pd.read_csv("./reoutputs/s1_re_total_inference_X.csv")
# print(total_inference_X)

# standardization
mins = total_inference_X.min(axis=0)
maxs = total_inference_X.max(axis=0)
total_inference_X = max_min_scale(total_inference_X, mins, maxs)

X = total_inference_X[total_inference_df["flag"]==1]
target = total_inference_df[total_inference_df["flag"]==1]["Rooftop_area"].values

# modeling
rfr = RandomForestRegressor(n_estimators=100, min_samples_split=2, n_jobs=-1, random_state=1212)
# rfr = RandomForestRegressor(n_estimators=100, min_samples_split=2, n_jobs=-1)
rfr.fit(X, target)
total_inference_df["inference"] = rfr.predict(total_inference_X)
# total_inference_df.to_csv("./outputs/extrapolation_results.csv", index=None)

# 将城市名相同的行进行合并，更换单位，并将小数点修改为后两位，保存为rooftop_area_360
rooftop_area_df=total_inference_df.groupby('City').agg({'inference':'sum'})/ 1e6
rooftop_area_df['Rooftop_area'] = rooftop_area_df['inference'].round(2)

total_inference_df_test = pd.read_csv("./inputs/Grid feature statistics.csv")

# 循环100次、
times=1001
for i in range(1, times):
    # modeling
    print(i)
    rfr = RandomForestRegressor(n_estimators=100, min_samples_split=2, n_jobs=-1)
    rfr.fit(X, target)
    inference_result = pd.DataFrame({i: rfr.predict(total_inference_X)})   
    total_inference_df_test = pd.concat([total_inference_df_test, inference_result], axis=1)

sum_by_city_million = total_inference_df_test.groupby('City').agg({i: 'sum' for i in range(1, times)}) / 1e6
sum_by_city_million = sum_by_city_million.round(2)
sum_by_city_million['Rooftop_area_initial'] = rooftop_area_df['inference'].round(2)
sum_by_city_million.to_csv("./reoutputs/s2_re_rooftop_area_360.csv")

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [4]:
import numpy as np
import pandas as pd

# S3: Read the list of city names and standardize all the names of the cities. Read the new list of city names, and standardize the non-standardized "City" and "City_en" in the "rooftop_area_360" section.

nameList = pd.read_csv("./inputs/city_name.csv")
# nameList_new = pd.read_csv("./city_name_new.csv")
for index, row in nameList.iterrows():
    if(row["City_en"]=="Ningbo"):
        nameList.loc[index,"City_en"] = 'Ningpo'
    if(row["City_en"]=="Xi’an"):
        nameList.loc[index,"City_en"] = 'Xi‘an'
    if(row["City_en"]=="Changzhou"):
        nameList.loc[index,"City_en"] = 'Chanzhou'
    if(row["City_en"]=="Ding’an"):
        nameList.loc[index,"City_en"] = 'Dingan'
    if(row["City_en"]=="Guang’an"):
        nameList.loc[index,"City_en"] = 'Guangan'
    if(row["City_en"]=="Ji’an"):
        nameList.loc[index,"City_en"] = 'Ji‘an'
    if(row["City_en"]=="Jinhua"):
        nameList.loc[index,"City_en"] = 'Jinghua'
    if(row["City_en"]=="Kizilsu Kirghiz"):
        nameList.loc[index,"City_en"] = 'Kizilsukirgiz'
    if(row["City_en"]=="Ma’anshan"):
        nameList.loc[index,"City_en"] = 'Maanshan'
    if(row["City_en"]=="Tai’an"):
        nameList.loc[index,"City_en"] = 'Taian'
    # if(row["City_en"]=="Urumqi"):
    #     nameList.loc[index,"City_en"] = 'Urumqi '
    # if(row["City_en"]=="Suzhou(Su)"):
    #     nameList.loc[index,"City_en"] = 'Suzhou(Su) '
    # if(row["City_en"]=="Hohhot"):
    #     nameList.loc[index,"City_en"] = 'Hohhot '
# nameList["City_en"]=nameList["City_en"].map(str.strip)
nameList.replace('\s+','',regex=True,inplace=True) 
# nameList.to_csv("city_name_new.csv")

# Add the roof areas of 85 real cities to the CSV file containing the roof areas of 360 calculated buildings
# Change the names
# rooftop_area_df = pd.read_csv("./outputs/s2_rooftop_area_360_initial.csv")
rooftop_area_df = pd.read_csv("./reoutputs/s2_re_rooftop_area_360.csv")
for index_city, row_city in rooftop_area_df.iterrows():
    for index, row in nameList.iterrows():
        if(row_city["City"]==row["城市"] ):
            rooftop_area_df.loc[index_city,"Province_en"] = row["Province_en"]
            rooftop_area_df.loc[index_city,"City_en"] = row["City_en"]
        if(row_city["City"]=='吐鲁番市'):
            rooftop_area_df.loc[index_city,"Province_en"] = 'Xinjiang'
            rooftop_area_df.loc[index_city,"City_en"] = 'Tulufan'
        if(row_city["City"]=='哈密市'):
            rooftop_area_df.loc[index_city,"Province_en"] = 'Xinjiang'
            rooftop_area_df.loc[index_city,"City_en"] = 'Hami'
        if(row_city["City"]=='新疆省直辖县行政区划单位'):
            rooftop_area_df.loc[index_city,"Province_en"] = 'Xinjiang'
            rooftop_area_df.loc[index_city,"City_en"] = 'XinjiangCorps'
        if(row_city["City"]=='昌都市'):
            rooftop_area_df.loc[index_city,"Province_en"] = 'Tibet'
            rooftop_area_df.loc[index_city,"City_en"] = 'Nyingchi'
        if(row_city["City"]=='林芝市'):
            rooftop_area_df.loc[index_city,"Province_en"] = 'Tibet'
            rooftop_area_df.loc[index_city,"City_en"] = 'Qamdo'
        if(row_city["City"]=='那曲市'):
            rooftop_area_df.loc[index_city,"Province_en"] = 'Tibet'
            rooftop_area_df.loc[index_city,"City_en"] = 'Nagqu'
        if(row_city["City"]=='林芝市'):
            rooftop_area_df.loc[index_city,"Province_en"] = 'Tibet'
            rooftop_area_df.loc[index_city,"City_en"] = 'Qamdo'
        if(row_city["City"]=='塔城地区'):
            rooftop_area_df.loc[index_city,"Province_en"] = 'Xinjiang'
            rooftop_area_df.loc[index_city,"City_en"] = 'Tarbagatay'

        
df_clear = rooftop_area_df.drop(rooftop_area_df[(rooftop_area_df['Province_en']=='Tibet') | 
                                                (rooftop_area_df['Province_en']=='Hongkong') | 
                                                (rooftop_area_df['Province_en']=='Macau')| 
                                                (rooftop_area_df['City_en']=='Macau')].index)
df_clear.replace('\s+','',regex=True,inplace=True) 

# df_clear.to_csv("./outputs/s3_rooftop_area_360_filterName.csv")
df_clear.to_csv("./reoutputs/s3_re_rooftop_area_360_filterName.csv")

In [5]:

# Assign the actual roof area to the "rooftop_area_360_filterName" and save it as "rooftop_area_addreal"

df_Supplementary = pd.read_excel("./inputs/Supplementary Data 1.xlsx", skiprows=1)
x_real_df = df_Supplementary[['City','Sample City', 'Rooftop Area (km2)','Province/Municipality']]
x_real_df.replace('\s+','',regex=True,inplace=True)

rooftop_area_df = pd.read_csv("./reoutputs/s3_re_rooftop_area_360_filterName.csv")

for index_city, row_city in rooftop_area_df.iterrows():
    # rooftop_area_df.loc[index_city,"Sample"] = "No"
    for index, row in x_real_df.iterrows():
        if(row_city["City_en"]==row["City"]):
            rooftop_area_df.loc[index_city,"Province_en"] = row["Province/Municipality"]
            if(row["Sample City"]=="Yes"):
                rooftop_area_df.loc[index_city,"Rooftop_area"] = row["Rooftop Area (km2)"]
                rooftop_area_df.loc[index_city,"1":"1000"] = row["Rooftop Area (km2)"]
                rooftop_area_df.loc[index_city,"Sample"] = "Yes"
            else:
                rooftop_area_df.loc[index_city,"Sample"] = "No"
            
rooftop_area_df.to_csv("reoutputs/s4_re_rooftop_area_withreal.csv")

print(rooftop_area_df)


/var/folders/y6/36h2pwl51ql85zwkh35jmm1r0000gp/T/ipykernel_62195/1581518537.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_real_df.replace('\s+','',regex=True,inplace=True)


     Unnamed: 0         City       1       2       3       4       5       6  \
0             0         七台河市   40.87   41.15   40.88   40.92   40.68   40.91   
1             1          万宁市   33.37   33.78   33.38   33.62   33.61   33.34   
2             2          三亚市   32.64   32.64   32.64   32.64   32.64   32.64   
3             3          三明市  107.93  107.92  107.36  107.82  107.87  107.92   
4             4          三沙市    0.64    0.69    0.64    0.69    0.73    0.69   
..          ...          ...     ...     ...     ...     ...     ...     ...   
349         355   黔东南苗族侗族自治州  152.45  152.46  152.96  151.80  151.85  152.00   
350         356   黔南布依族苗族自治州  184.18  184.67  184.93  184.73  183.82  184.67   
351         357  黔西南布依族苗族自治州  136.46  136.89  136.27  136.60  136.32  136.90   
352         358        齐齐哈尔市  235.83  235.83  235.83  235.83  235.83  235.83   
353         359          龙岩市  154.64  155.31  154.98  155.17  155.35  155.41   

          7       8  ...     996     99

In [6]:
import numpy as np
import pandas as pd
# # Calculate the Photovoltaic Grid Emission Factor (gCO2/Kwh)

rooftop_area = pd.read_csv("./reoutputs/s4_re_rooftop_area_withreal.csv")

North=['Beijing','Tianjin','Hebei','Shanxi','Shandong','InnerMongolia']
Northeast=['Liaoning','Jilin','Heilongjiang']
East=['Shanghai','Jiangsu','Zhejiang','Anhui','Fujian']
Central=['Henan','Hubei','Hunan','Jiangxi','Sichuan','Chongqing']
Northwest=['Shaanxi','Gansu','Qinghai','Ningxia','Xinjiang']
South=['Guangdong','Guangxi','Yunnan','Guizhou','Hainan']


North_factor= (0.9419 * 0.75+0.4819*0.25)*1000
Northeast_factor= (1.0826 * 0.75+0.2399*0.25)*1000
East_factor= (0.7921 * 0.75+0.3870*0.25)*1000
Central_factor= (0.8587 * 0.75+0.2854*0.25)*1000
Northwest_factor= (0.8922 * 0.75+0.4407*0.25)*1000
South_factor= (0.8042 * 0.75+0.2135*0.25)*1000

# print(North_factor,Northeast_factor,East_factor,Central_factor,Northwest_factor,South_factor)


for index_city, row_city in rooftop_area.iterrows():
    for row in North:
        if(row_city["Province_en"]==row):
            rooftop_area.loc[index_city,"GEFactor"] = North_factor
            rooftop_area.loc[index_city,"Region"] = "North"
    for row in Northeast:
        if(row_city["Province_en"]==row):
            rooftop_area.loc[index_city,"GEFactor"] = Northeast_factor
            rooftop_area.loc[index_city,"Region"] = "Northeast"
    for row in East:
        if(row_city["Province_en"]==row):
            rooftop_area.loc[index_city,"GEFactor"] = East_factor
            rooftop_area.loc[index_city,"Region"] = "East"
    for row in Central:
        if(row_city["Province_en"]==row):
            rooftop_area.loc[index_city,"GEFactor"] = Central_factor
            rooftop_area.loc[index_city,"Region"] = "Central"
    for row in Northwest:
        if(row_city["Province_en"]==row):
            rooftop_area.loc[index_city,"GEFactor"] = Northwest_factor
            rooftop_area.loc[index_city,"Region"] = "Northwest"
    for row in South:
        if(row_city["Province_en"]==row):
            rooftop_area.loc[index_city,"GEFactor"] = South_factor
            rooftop_area.loc[index_city,"Region"] = "South"
rooftop_area['GEFactor'] = rooftop_area['GEFactor'].round(2)
rooftop_area.to_csv("./reoutputs/s5_re_rooftop_area_emission.csv")
# print(rooftop_area)


In [10]:
import netCDF4 as nc
from osgeo import gdal, ogr
import rasterio
from rasterstats import zonal_stats
import time
def nctotif(nc_file, tiff_file, index):   
    # Open the NC file
    dataset = nc.Dataset(nc_file)

    # Obtain longitude, latitude and Rs data from the NC file
    longitude = dataset.variables['longitude'][:]
    latitude = dataset.variables['latitude'][:]
    Rs = dataset.variables['Rs'][:]
    
    # Obtain the grid size for latitude and longitude
    latitude_size = len(latitude)
    longitude_size = len(longitude)

    # Obtain the "nodata" value
    nodata_value = Rs[index].fill_value
    
    # Create a TIF file
    driver = gdal.GetDriverByName('GTiff')
    tif_ds = driver.Create(tiff_file, longitude_size, latitude_size, 1, gdal.GDT_Float32)

    # Set the geographic information of TIF
    tif_ds.SetGeoTransform((longitude[0], longitude[1] - longitude[0], 0, latitude[0], 0, latitude[1] - latitude[0]))
    tif_ds.SetProjection('EPSG:4326')

    # Written value

    band = tif_ds.GetRasterBand(1)
    band.WriteArray(Rs[index][:])
    band.FlushCache()
    band.ComputeStatistics(False)

   
    tif_ds = None
    dataset.close()
    
    return nodata_value
    
def area_statics(ras_path, shp_path, nodata_value):
    # Open the raster file
    ras_driver = rasterio.open(ras_path)

    # Read raster data as an array
    array = ras_driver.read(1)

    # 获取栅格的仿射变换参数
    affine = ras_driver.transform

    # 使用rasterstats库计算栅格数据的分区统计信息
    zs = zonal_stats(shp_path, array, affine=affine, nodata=nodata_value, stats="mean", all_touched = True)
    
    # 关闭文件
    ras_driver = None
    
    return zs
  
def shp_table(shp_path, field_name):
    ds = ogr.Open(shp_path)
    layer = ds.GetLayerByIndex(0)

    #
    field_values = []

    for feature in layer:
        field_value = feature.GetField(field_name)
        field_values.append(field_value)

    ds = None
    
    return field_values


In [12]:
#reproduced

start_time = time.time()

# Input NC
nc_file = r"./inputs/GWRHXG_Rs1.nc"

# Move the TIFF folder
tiff_folder = r"./reoutputs/new/"

# City SHP
shp_file = r"./inputs/China/China_2021_Districs_v3.shp"
field_name_list = shp_table(shp_file, "市")  # 输出字段名称

# Result output path
result_csv = r"./reoutputs/s6_re_output.csv"

# Convert NC format to TIFF format and perform regional statistics simultaneously
zs = [0] * 363

for i in range(282,402):
    tiff_file = fr"{tiff_folder}{i}.tif"
    nodata_value = nctotif(nc_file, tiff_file, i)
    temp_zs = area_statics(tiff_file, shp_file, nodata_value)
    for i in range(len(temp_zs)):
        if temp_zs[i]['mean'] == None: pass
        else: 
            zs[i] += float(temp_zs[i]['mean'])

# Sansha City Quota System
zs[field_name_list.index("三沙市")] = zs[field_name_list.index("三亚市")]

# Calculate the annual average surface solar radiation
annual_solar = zs
fcsv = open(result_csv, "w")
fcsv.write("City,Annual Radiation\n")
for i in range(0,len(zs)):
    annual_solar[i] = zs[i]/120*365*24/1000
    line = f"{field_name_list[i]},{annual_solar[i]}\n"
    fcsv.write(line)
    
fcsv.close()
# nctotif(nc_file, tiff_file)
# area_statics(tiff_file, shp_file)

end_time = time.time()
print(end_time-start_time)

259.585978269577


In [13]:
import numpy as np
import pandas as pd

radiation_results = pd.read_csv("./reoutputs/s6_re_output.csv")
nameList = pd.read_csv("./inputs/city_name_new.csv")
radiation_results['Radiation'] = radiation_results['Annual Radiation'].round(2)
print(radiation_results)

for index_city, row_city in radiation_results.iterrows():
    for index, row in nameList.iterrows():
        if(row_city["City"]==row["城市"] ):
            radiation_results.loc[index_city,"Province_en"] = row["Province_en"]
            radiation_results.loc[index_city,"City_en"] = row["City_en"]
        if(row_city["City"]=='吐鲁番市'):
            radiation_results.loc[index_city,"Province_en"] = 'Xinjiang'
            radiation_results.loc[index_city,"City_en"] = 'Tulufan'    
        if(row_city["City"]=='哈密市'):
            radiation_results.loc[index_city,"Province_en"] = 'Xinjiang'
            radiation_results.loc[index_city,"City_en"] = 'Hami'
        if(row_city["City"]=='新疆省直辖县行政区划单位'):
            radiation_results.loc[index_city,"Province_en"] = 'Xinjiang'
            radiation_results.loc[index_city,"City_en"] = 'Xinjiang Corps'
        if(row_city["City"]=='昌都市'):
            radiation_results.loc[index_city,"Province_en"] = 'Tibet'
            radiation_results.loc[index_city,"City_en"] = 'Nyingchi'
        if(row_city["City"]=='林芝市'):
            radiation_results.loc[index_city,"Province_en"] = 'Tibet'
            radiation_results.loc[index_city,"City_en"] = 'Qamdo'
        if(row_city["City"]=='那曲市'):
            radiation_results.loc[index_city,"Province_en"] = 'Tibet'
            radiation_results.loc[index_city,"City_en"] = 'Nagqu'
        if(row_city["City"]=='林芝市'):
            radiation_results.loc[index_city,"Province_en"] = 'Tibet'
            radiation_results.loc[index_city,"City_en"] = 'Qamdo'
        if(row_city["City"]=='塔城地区'):
            radiation_results.loc[index_city,"Province_en"] = 'Xinjiang'
            radiation_results.loc[index_city,"City_en"] = 'Tarbagatay'


        
df_clear = radiation_results.drop(radiation_results[(radiation_results['Province_en']=='Tibet') | 
                                                (radiation_results['Province_en']=='Hongkong') | 
                                                (radiation_results['Province_en']=='Macau')| 
                                                (radiation_results['City_en']=='Macau')].index)
df_clear.replace('\s+','',regex=True,inplace=True) 
print(df_clear)
df_clear.to_csv("./reoutputs/s7_re_radiation_filterName.csv")

     City  Annual Radiation  Radiation
0     安庆市       1282.019118    1282.02
1     蚌埠市       1356.566390    1356.57
2     亳州市       1318.974121    1318.97
3     池州市       1283.081163    1283.08
4     滁州市       1353.763006    1353.76
..    ...               ...        ...
358   舟山市       1347.815997    1347.82
359   重庆市       1110.179416    1110.18
360  雄安新区       1339.794436    1339.79
361   辛集市       1397.832284    1397.83
362   定州市       1346.291684    1346.29

[363 rows x 3 columns]
     City  Annual Radiation  Radiation Province_en    City_en
0     安庆市       1282.019118    1282.02       Anhui     Anqing
1     蚌埠市       1356.566390    1356.57       Anhui     Bengbu
2     亳州市       1318.974121    1318.97       Anhui     Bozhou
3     池州市       1283.081163    1283.08       Anhui    Chizhou
4     滁州市       1353.763006    1353.76       Anhui    Chuzhou
..    ...               ...        ...         ...        ...
358   舟山市       1347.815997    1347.82    Zhejiang   Zhoushan
359   重庆市   

In [14]:
#repeoduced

import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
import warnings 

warnings.filterwarnings("ignore")

radiation_df = pd.read_csv("./reoutputs/s7_re_radiation_filterName.csv")
area_emission_df = pd.read_csv("./reoutputs/s5_re_rooftop_area_emission.csv")
rooftop_area_df=pd.read_csv("./reoutputs/s4_re_rooftop_area_withreal.csv")

for index, row in rooftop_area_df.iterrows():
    for index_r, row_r in radiation_df.iterrows():
        if(row_r["City_en"]==row["City_en"] ):
            rooftop_area_df.loc[index,"Radiation"] = row_r["Radiation"]
    for index_r, row_r in area_emission_df.iterrows():
        if(row_r["City_en"]==row["City_en"] ):
            rooftop_area_df.loc[index,"Emission"] = row_r["GEFactor"]

# print(area_emission_df)
# rooftop_area_df.to_csv("./outputs/s8_rooftop_area_emission_radiation.csv")

df_Supplementary = pd.read_excel("./inputs/Supplementary Data 1.xlsx", skiprows=1)
x_real_df = df_Supplementary[['Region','Province/Municipality','City','Sample City','Province/Municipality','Rooftop Area (km2)',
                              'Grid Emission Factor (gCO2/Kwh)','Annual Surface Solar Radiation (Kwh/m2)','cluster']]
# rooftop_area_df=pd.read_csv("./outputs/s8_rooftop_area_emission_radiation.csv")
names=['Region','Province/Municipality','City','Sample City','Province/Municipality','Annual Surface Solar Radiation (Kwh/m2)',
       'Grid Emission Factor (gCO2/Kwh)','Rooftop Area (km2)']
x_real_df.colums= names

for index_real, row_real in x_real_df.iterrows():
    for index, row in rooftop_area_df.iterrows():
        if(row_real["City"]==row["City_en"] ):
            x_real_df.loc[index_real,"Emission0"] = row["Emission"]
            x_real_df.loc[index_real,"Radiation0"] = row["Radiation"]
            for i in range(1, times):             
                x_real_df.loc[index_real,f"Rooftop_area{i}"] = row[f"{i}"]      
        if(row_real["City"]=="Urumqi "):
            if(row["City_en"]=="Urumqi"):
                x_real_df.loc[index_real,"Emission0"] = row["Emission"]
                x_real_df.loc[index_real,"Radiation0"] = row["Radiation"]
                for i in range(1, times):
                    x_real_df.loc[index_real,f"Rooftop_area{i}"] = row[f"{i}"]
        if(row_real["City"]=="Suzhou(Su) "):
            if(row["City_en"]=="Suzhou(Su)"):
                x_real_df.loc[index_real,"Emission0"] = row["Emission"]
                x_real_df.loc[index_real,"Radiation0"] = row["Radiation"]
                for i in range(1, times):
                    x_real_df.loc[index_real,f"Rooftop_area{i}"] = row[f"{i}"]
        if(row_real["City"]=="Hohhot "):
            if(row["City_en"]=="Hohhot"):
                x_real_df.loc[index_real,"Emission0"] = row["Emission"]
                x_real_df.loc[index_real,"Radiation0"] = row["Radiation"]
                for i in range(1, times):
                    x_real_df.loc[index_real,f"Rooftop_area{i}"] = row[f"{i}"]
        if(row_real["City"]=="Xinjiang Corps"):
            if(row["City_en"]=="XinjiangCorps"):
                x_real_df.loc[index_real,"Emission0"] = row["Emission"]
                x_real_df.loc[index_real,"Radiation0"] = row["Radiation"]
                for i in range(1, times):
                    x_real_df.loc[index_real,f"Rooftop_area{i}"] = row[f"{i}"]
     

# print(x_real_df)
x_real_df.to_csv("./reoutputs/s8_re_merge.csv", index=None)


In [17]:
# x_real_df = pd.read_csv("./reoutputs/s8_re_merge.csv")


# X = x_real_df[['Annual Surface Solar Radiation (Kwh/m2)', 'Rooftop Area (km2)', 'Grid Emission Factor (gCO2/Kwh)']].values
# X_std = (X - X.mean(axis=0)) / X.std(axis=0)
# kmeans = KMeans(n_clusters=4, init="k-means++", random_state=1212).fit(X_std)
# classes = kmeans.predict(X_std)
# x_real_df["pred0"] = classes
# x_real_df["pred0"] = x_real_df["pred0"].map({0:2, 1:1, 2:3, 3:4})
       
# for i in range(1, times):
#     X = x_real_df[[f"Rooftop_area{i}", 'Radiation0', 'Emission0']].values
#     # standardization
#     X_std = (X - X.mean(axis=0)) / X.std(axis=0)
#     kmeans = KMeans(n_clusters=4, init="k-means++", random_state=1212).fit(X_std)
#     classes = kmeans.predict(X_std)
#     x_real_df[f"pred{i}"] = classes
#     x_real_df[f"pred{i}"] = x_real_df[f"pred{i}"].map({0:2, 1:1, 2:3, 3:4})
#     x_real_df.to_excel("./reoutputs/s9_cluster.xlsx", index=None)

import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import os

x_real_df = pd.read_csv("./reoutputs/s8_re_merge.csv")

# pred0
X = x_real_df[['Annual Surface Solar Radiation (Kwh/m2)',
               'Rooftop Area (km2)',
               'Grid Emission Factor (gCO2/Kwh)']].values
X_std = (X - X.mean(axis=0)) / X.std(axis=0)
kmeans = KMeans(n_clusters=4, init="k-means++", random_state=1212).fit(X_std)
classes = kmeans.predict(X_std)
x_real_df["pred0"] = pd.Series(classes).map({0: 2, 1: 1, 2: 3, 3: 4}).to_numpy()

# pred1..pred(times-1)
for i in range(1, times):
    X = x_real_df[[f"Rooftop_area{i}", 'Radiation0', 'Emission0']].values
    X_std = (X - X.mean(axis=0)) / X.std(axis=0)
    kmeans = KMeans(n_clusters=4, init="k-means++", random_state=1212).fit(X_std)
    classes = kmeans.predict(X_std)
    x_real_df[f"pred{i}"] = pd.Series(classes).map({0: 2, 1: 1, 2: 3, 3: 4}).to_numpy()

# ✅ 只写一次
os.makedirs("./reoutputs", exist_ok=True)
x_real_df.to_csv("./reoutputs/s9_cluster.csv", index=False)